In [2]:
import pandas as pd
import geopandas as gpd
from shapely.wkt import loads

df = pd.read_parquet("../../data/parquet/caraga-davao_lcc_2023_0.parquet.gzip")

In [3]:
df["geometry"] = df["geometry"].apply(loads)
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:3125")

In [4]:
gdf["area_sqm"] = gdf.geometry.to_crs("EPSG:3857").area

In [5]:
gdf.describe()

,area_sqm
count,1.000000e+06
mean,1.760934e+04
std,5.233763e+06
min,1.007913e+02
25%,2.023990e+02
50%,7.055617e+02
75%,2.119625e+03
max,3.966725e+09


In [6]:
subset_gdf = gdf[gdf.area_sqm > 1_500_000]

In [1]:
subset_gdf.shape

NameError: name 'subset_gdf' is not defined

In [10]:
import dask_geopandas as dgpd
from dask.distributed import Client

client = Client()  # Connect to a Dask cluster

subset_buffer = dgpd.from_geopandas(
    subset_gdf, npartitions=10
)  # Convert GeoDataFrame to Dask GeoDataFrame

subset_buffer = subset_buffer.map_partitions(
    lambda gdf: gdf.buffer(-200)
)  # Apply buffer operation to each partition

subset_buffer = subset_buffer.compute()  # Compute the result

subset_buffer.head()  # Display the result

/Users/renflores/miniconda3/envs/onebase/lib/python3.12/site-packages/distributed/client.py:3245: UserWarning: Sending large graph of size 174.15 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [20]:
subset_bufffer = subset_gdf.buffer(-200)

In [ ]:
import folium

# Create a Folium map instance
map_instance = folium.Map()

# # Display the map
# map_instance

In [ ]:
gdf.loc[gdf.LCC == "Swamp forest, closed", "buffered_geometry_buffer"].explore(
    m=map_instance, fill_color="red"
)

In [ ]:
gdf.loc[gdf.LCC == "Swamp forest, closed", "geometry"].explore(m=map_instance)

In [ ]:
from typing import Union
from shapely.geometry import Polygon, MultiPolygon


def one_sided_poly_buffer(
    poly: Polygon, buffer_m: float
) -> Union[Polygon, MultiPolygon]:
    bound = poly.boundary
    # estimate how big the buffer zone will be
    poly_len = min(poly.length, buffer_m)
    buffer_area_estimate = buffer_m * poly_len
    # no need to buffer for small polygons
    if poly.area < buffer_area_estimate:
        buffered_bound = poly
    # buffer for sufficiently large polygons
    else:
        buffered_bound = bound.buffer(-buffer_m / 2, single_sided=True)
        # take the intersection to remove the outward spikes from a single sided buffer
        buffered_bound = buffered_bound.intersection(poly)
        # buffer it again to fill in inward spikes
        buffered_bound = buffered_bound.buffer(buffer_m / 2)
        # take intersection to remove outward spikes again
        buffered_bound = buffered_bound.intersection(poly)
    return buffered_bound

In [ ]:
gdf.to_crs("EPSG:3857", inplace=True)

In [ ]:
gdf.crs

In [ ]:
%%time
gdf["buffered_geometry_buffer"] = gdf["geometry"].apply(
    lambda x: one_sided_poly_buffer(x, 200)
)

In [ ]:
gdf["area_buffered"] = gdf["buffered_geometry_buffer"].to_crs("EPSG:3857").area

In [ ]:
gdf["diff"] = gdf["area_sqm"] - gdf["area_buffered"]

In [ ]:
gdf["diff"].describe()

In [ ]:
from shapely import Polygon
import geopandas as gpd

# Define the coordinates of the bounding box
min_lon = 120.9
max_lon = 121.2
min_lat = 14.4
max_lat = 14.8

# Create a Polygon object representing the bounding box
bbox = Polygon(
    [(min_lon, min_lat), (max_lon, min_lat), (max_lon, max_lat), (min_lon, max_lat)]
)

# Create a GeoDataFrame with the bounding box
gdf_bbox = gpd.GeoDataFrame(geometry=[bbox])

# Set the coordinate reference system (CRS) of the GeoDataFrame
gdf_bbox.crs = "EPSG:4326"

# Print the GeoDataFrame
print(gdf_bbox)

In [ ]:
from shapely.geometry import Polygon
import geopandas as gpd

In [ ]:
# Define the coordinates of the bounding box
min_lon = 120.9
max_lon = 121.2
min_lat = 14.4
max_lat = 14.8

# Create a Polygon object representing the bounding box
bbox = Polygon(
    [(min_lon, min_lat), (max_lon, min_lat), (max_lon, max_lat), (min_lon, max_lat)]
)

# Create a GeoDataFrame with the bounding box
gdf_bbox = gpd.GeoDataFrame(geometry=[bbox])

# Set the coordinate reference system (CRS) of the GeoDataFrame
gdf_bbox.crs = "EPSG:3857"

# Print the GeoDataFrame
print(gdf_bbox)

In [ ]:
from shapely.geometry import Polygon

# Create a negative buffer of gdf_bbox with a distance of 200m
gdf_bbox_explode = gdf_bbox.explode()
negative_buffer = gdf_bbox_explode.geometry.buffer(-200)

# Create a new GeoDataFrame with the negative buffer
gdf_negative_buffer = gpd.GeoDataFrame(geometry=[negative_buffer])

# # Set the coordinate reference system (CRS) of the GeoDataFrame
# gdf_negative_buffer.crs = 'EPSG:3857'

# Print the GeoDataFrame
print(gdf_negative_buffer)

In [ ]:
import geopandas as gpd

print(gpd.__version__)